# Prediction using Endpoint

In [1]:
import numpy as np
import boto3
import pickle

from sagemaker.tensorflow import TensorFlowPredictor

In [2]:
#name of the endpoint
endpoint_name = 'your_endpoint'

#name of the busket where all resources are stored
bucket_name = 'your_bucket_name'

In [3]:
#download pickle files of preprocessing steps

s3_bucket = s3 = boto3.resource('s3').Bucket(bucket_name)

s3_bucket.download_file("prepr_x_inside_s3.pkl", "prepr_x_from_s3.pkl")
prepr_x = pickle.load(open("prepr_x_from_s3.pkl", "rb"))

s3_bucket.download_file("prepr_z_inside_s3.pkl", "prepr_z_from_s3.pkl")
prepr_z = pickle.load(open("prepr_z_from_s3.pkl", "rb"))

s3_bucket.download_file("prepr_y_inside_s3.pkl", "prepr_y_from_s3.pkl")
prepr_y = pickle.load(open("prepr_y_from_s3.pkl", "rb"))

In [4]:
#predictor object

predictor = TensorFlowPredictor(endpoint_name)

In [5]:
# random data, just for the example
# remember to preprocess the data if needed(not in this case as its just for example)

x_sample = np.random.normal(size=(10, 14, 6))
z_sample = np.random.normal(size=(10, 12))

#rescaling the data
x_sample = prepr_x.transform(x_sample.reshape(x_sample.shape[0], -1)).reshape(x_sample.shape)
z_sample = prepr_z.transform(z_sample)


inputs = {
    'inputs': {
        'input_1': x_sample.tolist(),
        'input_2': z_sample.tolist()
    }
}


#predicting the results
results = predictor.predict(inputs)
results

{'outputs': [[-2.58965182],
  [-2.65436244],
  [-2.76843762],
  [-2.53111768],
  [-2.64681411],
  [-2.6505897],
  [-2.02187276],
  [-2.13893867],
  [-2.38097215],
  [-2.66825271]]}

In [6]:
raw_prediction = results['outputs']

# transforming the predictions to its original value
prediction = prepr_y.inverse_transform(raw_prediction)
prediction

array([[ 6.89966816],
       [ 5.46906551],
       [ 3.83076834],
       [ 9.22712789],
       [ 5.60887014],
       [ 5.53024859],
       [59.95721122],
       [42.76994953],
       [18.07849286],
       [ 5.27507051]])